In [23]:
import pandas as pd
import numpy as np
from geopy.distance import distance

In [2]:
data = pd.read_csv('../../../capstone_other/files_too_large_for_github/ML_table_with_availability_clusters.csv')

In [3]:
data = data[['dock_id', 'dock_name', 'latitude', 'longitude',
           'avail_bikes', 'tot_docks', 'avail_bikes_proportion']]

In [4]:
data = data.groupby('dock_id').apply(pd.DataFrame.sample, n=1).reset_index(drop=True) 

In [5]:
data

,dock_id,dock_name,latitude,longitude,avail_bikes,tot_docks,avail_bikes_proportion
0,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,18,62,0.290323
1,119,Park Ave & St Edwards St,40.696089,-73.978034,12,19,0.631579
2,120,Lexington Ave & Classon Ave,40.686768,-73.959282,10,19,0.526316
3,143,Clinton St & Joralemon St,40.692395,-73.993379,13,24,0.541667
4,144,Nassau St & Navy St,40.698399,-73.980689,4,19,0.210526
...,...,...,...,...,...,...,...
324,3505,Lexington Ave & E 127 St,40.805726,-73.936322,13,33,0.393939
325,3506,Lexington Ave & E 120 St,40.801307,-73.939817,3,29,0.103448
326,3507,Park Ave & E 124 St,40.804555,-73.939686,12,36,0.333333
327,3509,Lenox Ave & W 115 St,40.801194,-73.950074,15,31,0.483871


In [25]:
def manhattan_distance(start_lat, start_lon, end_lat, end_lon):
    dist = distance((start_lat, start_lon), (start_lat, end_lon)).miles + \
           distance((end_lat, end_lon), (start_lat, end_lon)).miles
    return dist

In [6]:
data_low = data[data['avail_bikes_proportion'] <= 1/3]

In [7]:
data_high = data[data['avail_bikes_proportion'] >= 2/3]

In [8]:
data_low['deficit'] = round((1/3 - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')

<ipython-input-8-ae1edfedde08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_low['deficit'] = round((1/3 - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')


In [9]:
data_high['surplus'] = round((data_high['avail_bikes_proportion'] - 2/3) * data_high['tot_docks']).astype('int')

<ipython-input-9-11911a3c46e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high['surplus'] = round((data_high['avail_bikes_proportion'] - 2/3) * data_high['tot_docks']).astype('int')


In [10]:
data_low = data_low.sort_values(by = 'deficit', ascending = False)

In [11]:
data_high = data_high.sort_values(by = 'surplus', ascending = False)

In [29]:
rebalancing_dict = {}

In [30]:
low_copy = data_low.copy()
high_copy = data_high.copy()

In [31]:
for low in low_copy.index:
    if low_copy.loc[low, 'deficit'] == 0:
        continue
    for high in high_copy.index:
        if high_copy.loc[high, 'surplus'] == 0:
            continue
        
        if manhattan_distance(low_copy.loc[low, 'latitude'], low_copy.loc[low, 'longitude'], 
                              high_copy.loc[high, 'latitude'], high_copy.loc[high, 'longitude']) < 0.5:
            stations_key = (low_copy.loc[low, 'dock_id'], high_copy.loc[high, 'dock_id'])
            change = min(low_copy.loc[low, 'deficit'], high_copy.loc[high, 'surplus'])
            low_copy.loc[low, 'deficit'] -= change
            high_copy.loc[high, 'surplus'] -= change
            if stations_key in rebalancing_dict.keys():
                rebalancing_dict[stations_key] += change
            else:
                rebalancing_dict[stations_key] = change
            if low_copy.loc[low, 'deficit'] == 0:
                break

In [32]:
rebalancing_dict

{(3161, 3163): 9,
 (3161, 3165): 2,
 (253, 168): 8,
 (253, 297): 4,
 (469, 3466): 3,
 (3096, 3093): 5,
 (2017, 516): 2,
 (3284, 3305): 8,
 (3153, 3139): 3,
 (439, 312): 2,
 (337, 260): 6,
 (337, 264): 6,
 (3359, 3137): 4,
 (3463, 491): 12,
 (316, 315): 3,
 (3379, 3425): 8,
 (3379, 3391): 3,
 (3147, 3148): 7,
 (3147, 3288): 4,
 (390, 3486): 8,
 (390, 310): 2,
 (3090, 2002): 6,
 (448, 3236): 9,
 (3258, 462): 8,
 (3103, 3109): 9,
 (3117, 3116): 5,
 (3117, 3469): 1,
 (397, 258): 3,
 (357, 369): 3,
 (3491, 3490): 7,
 (3491, 3497): 1,
 (3495, 3497): 1,
 (3495, 3498): 2,
 (3381, 3407): 6,
 (3381, 3397): 1,
 (3398, 3397): 4,
 (3398, 3395): 2,
 (3404, 3319): 6,
 (3404, 3399): 1,
 (3296, 3288): 1,
 (3456, 3454): 4,
 (364, 354): 2,
 (3384, 3321): 4,
 (3384, 3332): 1,
 (3496, 3391): 3,
 (396, 437): 5,
 (3309, 3351): 3,
 (3346, 3414): 4,
 (3304, 3365): 4,
 (3056, 3057): 4,
 (3067, 3179): 3,
 (83, 3414): 3,
 (325, 491): 3,
 (3063, 3062): 1,
 (3061, 3179): 2,
 (3124, 3122): 2,
 (3059, 3179): 2,
 (317

In [ ]:
rebalancing_dict = 

In [16]:
data_low.head(3)

,dock_id,dock_name,latitude,longitude,avail_bikes,tot_docks,avail_bikes_proportion,deficit
151,3161,W 76 St & Columbus Ave,40.780184,-73.977285,0,59,0.000000,20
144,3141,1 Ave & E 68 St,40.765005,-73.958185,0,59,0.000000,20
19,253,W 13 St & 5 Ave,40.735439,-73.994539,1,55,0.018182,17


In [17]:
data_high.head(3)

,dock_id,dock_name,latitude,longitude,avail_bikes,tot_docks,avail_bikes_proportion,surplus
81,491,E 24 St & Park Ave S,40.740964,-73.986022,52,53,0.981132,17
303,3467,W Broadway & Spring Street,40.724947,-74.001659,40,42,0.952381,12
169,3236,W 42 St & Dyer Ave,40.758985,-73.993800,32,33,0.969697,10


In [18]:
data_low

,dock_id,dock_name,latitude,longitude,avail_bikes,tot_docks,avail_bikes_proportion,deficit
151,3161,W 76 St & Columbus Ave,40.780184,-73.977285,0,59,0.000000,20
144,3141,1 Ave & E 68 St,40.765005,-73.958185,0,59,0.000000,20
19,253,W 13 St & 5 Ave,40.735439,-73.994539,1,55,0.018182,17
79,469,Broadway & W 53 St,40.763441,-73.982681,4,57,0.070175,15
198,3316,W 104 St & Amsterdam Ave,40.798994,-73.966217,1,47,0.021277,15
...,...,...,...,...,...,...,...,...
56,373,Willoughby Ave & Walworth St,40.693317,-73.953820,6,19,0.315789,0
109,3072,Leonard St & Boerum St,40.705833,-73.946446,8,24,0.333333,0
293,3453,Devoe St & Lorimer St,40.713352,-73.949103,6,19,0.315789,0
138,3134,3 Ave & E 62 St,40.763126,-73.965269,17,51,0.333333,0
